# Biological Models


Search for the optimal biological model.

----
```
author:     Zach Wolpe
email:      zachcolinwolpe@gmail.com
date:       21 January 2022
```
----

In [10]:
import sys
sys.path.append('../process data')
from dependencies import *

In [54]:
# load -------------------------------------*
loc = '/Users/zachwolpe/Documents/Production/Dynocog/Python Implementation/final instance/model-free analysis/final_dataframes'
wcst_data   = pd.read_pickle(loc + '/wcst_raw_data.pkl')
wcst        = pd.read_pickle(loc + '/wcst.pkl')
covariates  = pd.read_pickle(loc + '/covariates.pkl')
wcst

,participant,reward,status,action,rule,n_t
0,816404.0,1,1,1,shape,0
1,816404.0,1,1,1,shape,1
2,816404.0,1,1,1,shape,2
3,816404.0,1,1,1,shape,3
4,816404.0,1,1,1,shape,4
...,...,...,...,...,...,...
27395,684712.0,1,1,3,color,95
27396,684712.0,1,1,3,color,96
27397,684712.0,1,1,3,color,97
27398,684712.0,1,1,3,color,98


# Null Bio-Model

Single learning rate.

## Transform the data

In [84]:
wcst.action   = wcst.action.astype(int)
wcst.reward   = wcst.reward.astype(int)
action_matrix = wcst[['participant' ,'n_t', 'action']].pivot(index='participant', columns='n_t')
reward_matrix = wcst[['participant' ,'n_t', 'reward']].pivot(index='participant', columns='n_t')
data_object = {
    'n_s':    reward_matrix.shape[0],
    'n_t':    reward_matrix.shape[1],
    'action': action_matrix,
    'reward': reward_matrix+1
}


In [85]:
type(action_matrix.iloc[0,0])

numpy.int64

In [86]:

wcst.n_t

0         0
1         1
2         2
3         3
4         4
         ..
27395    95
27396    96
27397    97
27398    98
27399    99
Name: n_t, Length: 27400, dtype: int64

In [87]:
model = """

data {
  int<lower=1> n_s;
  int<lower=1> n_t;
  int action[n_s, n_t];     
  int reward[n_s, n_t]; 
}

transformed data {
  vector[3] initQs;  // initial values for V
  initQs = rep_vector(0.3333, 3);
}

parameters {
  real<lower=0,upper=1> a_mu;
  real<lower=0,upper=10> b_mu;

  real<lower=0> a_sd;
  real<lower=0> b_sd;

  real<lower=0,upper=1> a[n_s];
  real<lower=0,upper=10> b[n_s];  
}

model {
  // individual variance priors! 
  a_sd  ~ cauchy(0,1);
  b_sd ~ cauchy(0,3);
  
  // give the prior here: how individual-level parameters are connected to the group-level parameters
  a ~ normal(a_mu, a_sd);
  b ~ normal(b_mu, b_sd);
  
  
  for (s in 1:n_s) {
    vector[3] Qs; 
    real pe;    
    Qs = initQs;

    for (t in 1:n_t) {        
      action[s,t] ~ categorical_logit( b[s] * Qs);
      pe = reward[s,t] - 1 - Qs[action[s,t]];      
      Qs[action[s,t]] = Qs[action[s,t]] + a[s] * pe; 
    }
  }    
}


// include log-likelihood calculation
generated quantities {
  real                    log_lik[n_s];

  { // local section, this saves time and space
    for (s in 1:n_s) {
      vector[3] Qs; 
      real pe;    

      log_lik[s] = 0;
      Qs = initQs;

      for (t in 1:n_t) {    
        log_lik[s] = log_lik[s] + categorical_logit_lpmf(action[s,t] | b[s] * Qs);    
              
        pe = reward[s,t] - 1 - Qs[action[s,t]];      
        Qs[action[s,t]] = Qs[action[s,t]] + a[s] * pe; 
      }
    }    
  }
}

"""

fit   = pystan.stan(model_code=model, warmup=1000, data=data_object, iter=2000, chains=2)


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_d500a01348f558f4f52414388e38da00 NOW.
In file included from /var/folders/kd/p1z7_f_974d7tkw1zmvskn_m0000gn/T/pystan_udifyj1i/stanfit4anon_model_d500a01348f558f4f52414388e38da00_644543329292557634.cpp:771:
In file included from /Users/zachwolpe/.local/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /Users/zachwolpe/.local/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/zachwolpe/.local/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1944:
/Users/zachwolpe/.local/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
In file included from /var/folders/kd/p1z7_f_974d7tkw1zmvskn_m0000gn/T/pystan_udifyj

KeyboardInterrupt: 

In [ ]:
print(fit)

In [ ]:
loo, loos, ks = psisloo(fit['log_lik'])
print('PSIS-LOO value: {:.2f}'.format(loo))

In [ ]:

import arviz as az
az.plot_trace(fit);

In [ ]:
import plotly.express as px
print('Alpha: ', np.mean(fit['a_mu']))
px.histogram(x=fit['a_mu'], template='none', opacity=0.8,
                  #  log_y=True, # represent bars with log scale
                   color_discrete_sequence=['lightblue'])

# print('beta: ', np.mean(fit['b_mu']))
# px.histogram(x=fit['b_mu'], template='none')